# Notebook inställningar

## Installera beroenden

In [ ]:
!pip install numpy pandas tensorflow tensorflow_datasets matplotlib seaborn

# Importera beroenden och förbered miljön

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import tensorflow as tf
import tensorflow_datasets as tfds
#import tensorflow_data_validation as tfdv
from tensorflow.data import Dataset
import matplotlib.pyplot as plt
import seaborn as sns

def print_versions():
  print('tensorflow: ', tf.__version__)
  print('tensorflow_datasets: ', tfds.__version__)

print_versions()

# Ladda och undersök träningsdata

Vi laddar ner öppen data från https://www.tensorflow.org/datasets/catalog/malaria.


## Kontroll att data stämmer med dokumentation
En utskrift av _features_ och _splits_ förväntas ge
```
info.features= FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
})
info.splits= {'train': <SplitInfo num_examples=27558, num_shards=4>}
```

Detta motsvarar våra förväntning att datan består av en träningsmängd bestående av numeriskt märkta bilder.
Denna datamängd saknar uppdelning i träning, test, validering så vi får själva ta fram en strategi för detta senare.

In [ ]:
ds, info = tfds.load('malaria', split='train', shuffle_files=True, with_info=True)
print('info.features=', info.features)
print('info.splits=', info.splits)

# Kontroll att data är balanserad
Data består av bilder av blodceller märkta med antingen _parasitized_ eller _uninfected_.

In [ ]:
def get_human_readable_labels():
  return info.features['label'].names

get_human_readable_labels()

Ett histogram grupperat på märkning säger oss att datan är balanserad.

In [ ]:
# Vi använder DataFrame för dess förmåga att gruppera och plotta med enkel syntax
def get_histogram(sample_count = -1):
  return (DataFrame({
    'label': [d['label'].numpy() for d in ds.take(sample_count)]
    })['label']
    .value_counts()
    .plot.bar(title='Fördelning av antal bilder grupperat på märkning'))
get_histogram()

## Analys av bildata
Vi undersöker bilderna med avseende på olika kvalitetsmått (bredd/höjd, kondningsstorlek, färg) i syfte att upptäcka
- skillnader mellan bilder grupperat på olika märkningar. Detta antyder ett systematiskt fel i skapandet av datamängden.
- vilka typer av storleksvariationer som förekommer i syfte att planera korrekt normalisering av data.


### Stickprovskontroll att bilderna ser bra ut för ögat
Utifrån en okulär granskning av några av bilderna kan vi dra slutsaten att
- bilderna är i färg
- bilderna varierar i färg även om enskilda bilder håller sig till en grundfärg
- bilderna är väl beskurna
- bilderna inte innehåller annoteringar som kan påverka maskininlärning (exv studie, patientnummer,...)
- bilderna varierar i storlek
- inte alla bilder är kvadratiska


In [ ]:
  tfds.show_examples(ds, info)

### Kontroll av bildstorlek
Om träningsdata varierar eller innehåller bias med avseende på bildstorlek kan det krävas normalisering innan träning.

#### Bildkodning
Vi tittar på hur 1:a bilden är kodad.

In [ ]:
for image in ds.take(1):
  print(image['image'])

Utmatningen ovan ger att bildata är kodat som multidimensionella vektorer/tensorer.
#### Fördelning bildtyngd
Vi tittar på fördelning över hur mycket minne enskilda bilder tar i anspråk.
- beräkningen är grupperad på märkning i syfte att upptäcka skillnader i systematik för olika fall
- det räcker med ett mindre urval för vidare slutsatser

In [ ]:
def get_image_size_analysis(ds: tf.data.Dataset, sample_size: int):
  labels = [d['label'].numpy() for d in ds.take(sample_size)]
  image_lengths = [len(d['image'].numpy().flatten()) for d in ds.take(sample_size)]

  return DataFrame({
      'label': labels,
      'size': image_lengths
  }).groupby('label').describe()

get_image_size_analysis(ds, 10000)

Analysen ovan visar att bildtyngd varierar mer än önskat och att normalisering blir nödvändig.
- standardavvikelsen uppgår till mer än 1/3 av medelvärdet
- minsta och största skiljer sig med mer än en faktor 30 (label=0)

#### Fördelning bredd/höjd
Vi utför samma analys med avseende på bredd och höjd (på ett urval av datamängden).

In [ ]:
def get_image_dimension_analysis(ds: tf.data.Dataset, sample_size: int):
  image_shapes = [d['image'].shape for d in ds.take(sample_size)]
  return DataFrame({
      'image width': [s[0] for s in image_shapes],
      'image height': [s[1] for s in image_shapes],
      'aspect ratio': [s[0]/s[1] for s in image_shapes]
  }).describe()

get_image_dimension_analysis(ds, 10000)


Vi ser att bilderna varierar i höjd och bredd och även om bilderna i medeltal är nästan kvadratiska utgörs allför många av olika rektanglar inom intervallet (bredd, höjd) = [(79, 61) ... (232, 250)].

För att inte överträna modellen måste vi hitta ett sätt att skala om alla bilder till samma dimensioner.

### Kontroll av färgpalett

In [ ]:
for d in ds.take(1):
  tf.image.total_variation(d['image'].numpy())

#tf.image.total_variation(ds.take(1), name='image')
